---
title: Evaluating causality in observational data
author: Erdem Karaköylü
date: '2022-01-15'
categories:
  - Bayesian A/B/n
  - pivoting
  - categorical regression
  - ZeroSumNormal
  - Python
image: ''
format:
  html:
    code-fold: true
jupyter: python3
---



<u>**Preamble**</u>: This is a data analysis and modeling project to predict voter party preference in Estonia on the basis of demographics. The intended process involves initial data exploration, followed by the construction, fitting, and analysis of a series of increasingly complex Bayesian categorical models. This Bayesian approach offers a robust alternative to traditional frequentist Analysis of Variance (ANOVA) methods commonly employed in scientific research and A/B/n testing in fields such as marketing, user experience (UX), and web design. A significant benefit of the Bayesian model is its inherent ability to quantify predictive uncertainty through the calculation of posterior (inferential) probability distributions, contrasting with the reliance on sampling distributions in frequentist procedures

<u>**Contents**</u>:

1. Exploratory Data Analysis
2. Modeling
    <br> $→$ Model Building and, Prior Elicitation and Prior Predictive Checks
    <br> $→$ Model Fitting and Goodness-of-Fit Analysis
    <br> $→$ Posterior Predictive Checks
    <br> $→$ Summary and Conclusion

 <u>**Data Provenance**</u>: This data is courtesy of [SALK](https://salk.ee/). SALK refers to the Liberal Citizen Foundation in Estonia; a political organization aimed at influencing Estonian policy and parliamentary elections. This foundation was established with the (ultimately attained) goal of helping liberal forces gain a majority in the 2023 Estonian parliamentary elections.

<u>**Acknowledgments**</u>: I thank [Alex Andorra](https://topmate.io/alex_andorra) and the folks at [Intuitive Bayes](https://www.intuitivebayes.com/) for helping me understand non-identifiability and overparameterization in statistical modeling.


In [ ]:
import arviz as az
import matplotlib.pyplot as pp
from matplotlib import rcParams
import numpy as np
import pandas as pd

import pymc as pm

In [ ]:
rcParams['font.size'] = 12

#### 1. Exploratory Data Analysis


In [ ]:
data = pd.read_csv('~/projex/elections/data/estonian-data.csv')

In [ ]:
data.head().T

I don't like the "Hard to say" column name; replace below with the clearer "Undecided" label.


In [ ]:
data.rename(columns={'Hard to say': 'Undecided'}, inplace=True)

In [ ]:
data.info()

In [ ]:
data.columns.tolist()

In [ ]:
data.describe().T

In [ ]:
data.loc[:, 'EKRE':].sum(axis=1).describe().loc[['min', 'max']]

In [ ]:
data.loc[:, 'EKRE':].sum(axis=0).plot(kind='bar', title='Party Choice', ylabel='Counts', color='black', alpha=0.6);

In [ ]:
data.electoral_district.unique().size, data.unit.unique().size

In [ ]:
f, (left, right) = pp.subplots(ncols=2, figsize=(12, 6), sharey=True)
data.electoral_district.value_counts(normalize=True).plot(kind='bar', color=['black', 'darkgray'], ax=left)
data.unit.value_counts(normalize=True).plot(kind='bar', color=['darkgray', 'black'])
left.set_xticklabels(left.get_xticklabels(), rotation=90)
f.tight_layout();

In [ ]:
f, (left, right) = pp.subplots(ncols=2, figsize=(10, 4), sharey=True)
data.age_group.value_counts(normalize=True, sort=False).plot(kind='bar', color=[f'C{i}' for i in range(7)], ax=left);
data.gender.value_counts(normalize=True).plot(kind='bar', color= ['C7', 'C8'], ax=right)
left.set(ylabel='Fraction', xlabel='age group')
f.tight_layout()

In [ ]:
data.education.value_counts()

In [ ]:
f, (left, right) = pp.subplots(ncols=2, figsize=(10, 4), sharey=True)
data.education.value_counts(normalize=True).plot(kind='bar', color=[f'C{i}' for i in range(0, 3)], ax=left)
data.nationality.value_counts(normalize=True).plot(kind='bar', color=['C3', 'C4'])
left.set_xticklabels(left.get_xticklabels(), rotation=50)
f.tight_layout();

##### Preliminary summary:

This table registers voter preference. Each row is a voter. Columns include voter demographics, the election district and unit. The rest of the columns are each dedicated to a party.  Data are in the thousands with no apparent missing value.
There are two types of data; (1) strings that lend themselves to categorization, and (2) binary data indicating whether a given party has received the vote of that row's  voter. There were no irregularities, i.e. more than 1 vote registered for each person;  there were no abstentions noted in this dataset either. 
All the data is categorical with the following number of categories:
* Party choice $→$ 11
* Electoral district $\rightarrow$ 12
* (Electoral) Unit $→$ 24
* AGe group $→$ 7
* Gender $→$ 2
* Education $→$ 3
* Nationality $→$ 2


##### Preliminary summary, part 2: 
There is some imbalance in education level, with secondary education being the more proeminent. There is notable imbalance in nationality, with predictable majority of Estonian representation. Party choice, electoral district and unit are also quite imbalanced. Next is to examine party representation breakdown at the  group level. Note that I will not consider electoral district and unit hereafter as the emphasis is on demographics as a driver of party choice.


In [ ]:
def make_group_percentage(df: pd.DataFrame, category: 'str') -> pd.DataFrame:
    """Computes percentage for each column in a group"""
    group = df.groupby(category).sum(numeric_only=True)
    group_percent = group.div(group.sum(axis=1), axis=0)*100
    return group_percent

In [ ]:
party_names = data.loc[:, 'EKRE':].columns.tolist()
party_names

In [ ]:
data.head()

In [ ]:
data.groupby('gender').sum(
numeric_only=True
)

In [ ]:
f, (left, right) = pp.subplots(ncols=2, figsize=(11, 4), sharey=True)
#gender = data.groupby('gender').sum(numeric_only=True)
#(gender.div(gender.sum(axis=1), axis=0)*100).plot(kind='bar', colormap='tab20c', ax=left, legend=False)
#data.groupby('nationality').sum(numeric_only=True).plot(kind='bar', colormap='tab20c', ax=right, legend=False)
make_group_percentage(data, 'gender').plot(kind='bar', colormap='tab20c', ax=left, legend=False)
make_group_percentage(data, 'nationality').plot(kind='bar', colormap='tab20c', ax=right)
left.set(title='Gender', ylabel='Fraction(%)', xlim=(-0.3, 1.3), xlabel='')
right.set(xlim=(-0.3, 1.3), title='Nationality', xlabel='')
right.legend(frameon=False, loc=(0.34 ,0.04), fontsize=10)
right.grid(axis='y', alpha=0.5, ls=':')
left.grid(axis='y', alpha=0.5, ls=':')
f.tight_layout();

In [ ]:
f, (top, bottom) = pp.subplots(nrows=2, figsize=(12, 8))
#data.groupby('age_group').sum(numeric_only=True)
make_group_percentage(data, 'age_group').plot(kind='bar', colormap='tab20c', ax=top, legend=False)
#data.groupby('education').sum(numeric_only=True)
make_group_percentage(data, 'education').plot(kind='bar', colormap='tab20c', ax=bottom, legend=False, )
top.set(title='Age Group', ylabel='Fraction(%)', xlabel='')
top.set_xticklabels(top.get_xticklabels(), rotation=0)
bottom.legend(frameon=False,  loc=(0.24, 0.04), fontsize=10)
bottom.set_xticklabels(bottom.get_xticklabels(), rotation=0)
bottom.set(ylabel='Fraction(%)', title='Education', xlim=(-0.3, 2.3), xlabel='')
bottom.grid(axis='y', alpha=0.5, ls=':')
top.grid(axis='y', alpha=0.5, ls=':')
f.tight_layout();

#### Grouped data summary
1. Overall
    * Reformierakond (a liberal party) is pretty popular across the board except with non-Estonians; 
    * Non-Estonians - I'm told - is almost entirely composed up of Russians;
    * There is a fair amount of undecided voters;
    * EKRE and Keskeradon are populist parties - EKRE is right-wing, Keskerakond is center-left;
    * Parempoolsed, center-right party is the underdog across the board.

2. Group-specific
    * *Gender*: there are some subtle  differences. 
        * Reformierakond is slightly more popular with female voters;
        * There are more undecided among female voters;
        * The right-wing EKRE is more popular with male voters.
    * *Nationality*:
        * Reformierakond, EKRE are more popular with Estonians;
        * Keskerakond, Mitte ukski erakond are favored by ethnic Russians who are also more numerous to be undecided. 
    * *Age group*:
        * Mitte ukski erakond and Reformierakond are the two prevaiing parties among each age group except the two older tranches;
        * Among 65 and over Reformierakond retains the top but Mitte recedes in favor of Keskeradond.
    * *Education*:
        * EKRE, Undecided, Mitte ukski erakond are the top 3 party choices for those with a basic education;
        * Reformierakond and Mitte are the top 2 party choices among those with a college degree.
        * High school graduates exhibit a preference for Reformierakond, Mitte, and EKRE.

#### 2. Modeling

The goal is to predict party preference on the basis of available demographic data. 
I will first start with looking at two categories; education and nationality. The below is some data encoding to ease model fitting and clarity of plotted results. In particular note the model coordinate names; 'party_choice', 'education', 'nationality', 'obs_idx'. The latter is just the row index of the dataframe containing the observations; each row corresponding to one observation.


In [ ]:
# response variable encoding
party_choice = pd.Categorical(data[party_names].idxmax(axis=1), categories=party_names, ordered=True)
party_choice_label = party_choice.categories.to_list()
party_choice_code = party_choice.codes

# input variables encoding
education_code, education_label = data.education.factorize(sort=True) 
nationality_code, nationality_label = data.nationality.factorize(sort=True)

# setting model coordinates

coords = {
    'party_choice': party_choice_label,
    'education': education_label,
    'nationality': nationality_label,
    'obs_idx': data.index} # observation index, the row location of an observation in the dataframe.

Below I write a first model using a baseline intercept and I use nationality and education as predictors and baseline offsets. The corresponding coefficients have 'party choice' as dimension (11 possibilities) and in the case of education and nationality, have  also a dimension of 'education' (3 possibilities) and 'nationality' (2 possibilities), respectively. These are linearly combined and passed to a softmax function to scale the output to the $(0-1)$ interval, as shown below.
$$ p = softmax(α_{baseline} + α_{nationality}[\small{nationality\_category}] + α_{education}[\small{education\_category}]) $$

As prior distribution for all three coefficients, instead of using the $\mathcal{Normal}$ distribution, I use below the $\mathcal{Zero-sum\ Normal}$. This distribution imposes zero sum constraints on specified axes of the fitted coefficients, thereby addressing the problem of overparameterization and identifiability that often plague categorical regression models; a subject of my blog post found [here](insert link). 

Finally I use for likelihood the $\mathcal{Categorical}$ distribution, which is a n-dimensional $\mathcal{Bernoulli}$ process, and through which I expose the model to the data.


In [ ]:
def build_model1(α_sigma=1, α_nat_sigma=1, α_edu_sigma=1):
    with pm.Model(coords=coords) as model1:
        # Data containers
        party_choice_idx = pm.Data('party_choice_index', party_choice_code, dims='obs_idx')
        nationality_idx = pm.Data('nationality_index', nationality_code, dims='obs_idx')
        education_idx = pm.Data('eductation_index', education_code, dims='obs_idx')
        
        # Model priors
        # baseline
        α = pm.ZeroSumNormal('α', sigma=α_sigma, dims='party_choice') 
        # baseline offset due to nationality
        α_nationality = pm.ZeroSumNormal('α_nationality', sigma=α_nat_sigma, dims=('nationality', 'party_choice'), n_zerosum_axes=2) 
        # baseline offset due to education
        α_education = pm.ZeroSumNormal('α_education', sigma=α_edu_sigma, dims=('education', 'party_choice'), n_zerosum_axes=2)

        # Link function for choice probability
        p = pm.math.softmax(α + α_nationality[nationality_idx] + α_education[education_idx], axis=-1)
        
        # Likelihood
        _ = pm.Categorical('y', p=p, observed=party_choice_idx, dims='obs_idx')
    return model1

In [ ]:
model1 = build_model1()

Below is the model's diagram with dimension information included. 


In [ ]:
model1.to_graphviz()

Next is to sample model priors to see the soundness of assumptions made


In [ ]:
with model1:
    idata1 = pm.sample_prior_predictive(model=model1)
    idata1.extend(pm.sample(chains=4, draws=2000, random_seed=42))
    idata1.extend(pm.sample_posterior_predictive(idata1))

In [ ]:
def plot_forward_samples(
        idata: az.InferenceData, categories:list, categories_name:str, 
        figsize=(12, 6), plot_prior=True, plot_posterior=True):
    """A quick function to plot model predictives. """
    f, (left, right) = pp.subplots(1, 2, figsize=figsize, sharey=True)
    xticks = [i + 0.5 for i in range(len(categories))]
    if plot_prior:
        az.plot_ppc(idata, group="prior", ax=left)
        left.set(
            xticks=xticks,
        )
        left.set_xticklabels(categories, fontsize=10, rotation=30)
        left.set_xlabel(categories_name, fontsize=16)
        left.legend(frameon=True, fontsize=11)
    else:
        left.set_visible(False)

    if plot_posterior:
        az.plot_ppc(idata, ax=right)
        right.set(
            xticks=xticks,
        )
        right.set_xticklabels(categories, fontsize=10, rotation=30)
        right.set_xlabel(categories_name, fontsize=16)
        right.legend(frameon=True, fontsize=11)
    else:
        right.set_visible(False)
    f.tight_layout()
    return f, (left, right)

In [ ]:
mean_win = pd.Series(party_choice_code).value_counts(normalize=True).round(3).sort_index()
f, (axl, axr) = plot_forward_samples(idata1, categories=party_choice_label, categories_name='party choice', figsize=(12, 6))
axl.set_xticklabels(axl.get_xticklabels(), rotation=60);
axl.bar(x=axl.get_xticks(), height=mean_win.values,  color='k', fill=False, lw=3, width=1, zorder=13, label='data');
axr.set_xticklabels(axr.get_xticklabels(), rotation=60);

Left, in blue lines are sample outcomes for each party choice, before the model has seen the data. Right, model fit results (posterior distribution for each category). The model seems to have learned the posterior distribution well. The model did take some time to fit though and in spite of the lack of difergences, one wonders if sampling could be more efficient. Domain experts have suggested a tighter variance on the coefficient priors could indeed ease sampling. Model1b integrates this domain knowledge as follows: 


In [ ]:
model1b = build_model1(α_sigma=0.5, α_edu_sigma=0.2, α_nat_sigma=0.3)

In [ ]:
with model1b:
    idata1b = pm.sample_prior_predictive()
    idata1b.extend(pm.sample(chains=4, draws=2000, random_seed=42))
    idata1b.extend(pm.sample_posterior_predictive(idata1b))

There was some speed improvement (~ 18 seconds on a Macbook air M2.)
#### Assessing goodness of fit


In [ ]:
az.plot_trace(idata1, backend_kwargs={'tight_layout': True});

In [ ]:
az.plot_trace(idata1b, backend_kwargs=dict(tight_layout=True));

Good mixing in the chains and lack of divergences suggest a good fit. The second model (bottom panel) has tighter posteriors, suggesting a slightly better fit.


In [ ]:
az.summary(idata1, var_names='α')

In [ ]:
az.summary(idata1b, var_names='α')

Looking at the model fit summary tables for the baseline $α$ parameter, in both cases the $\hat{R}$ (aka *Gelman-Rubin* statistic) is 1.0. This indicates good convergence of MCMC chains (I fired up 4 of them for sampling). Interestingly however, in the case of the second model with tighter variance constraints on the priors, the *Effective Sample Size* corresponding to the sampling of the central portion of the posterior (ess_bulk) shows more efficient sampling in most cases. The posterior is therefore likelier to be better characterized. A similar pattern can be observed in the next 4 tables for $α_{nationality}$ and $α_{education}$. Note that in contrast, the tails of the posterior are sampled with similar efficiency in both cases, as noted by ess_tail. 


In [ ]:
az.summary(idata1, var_names='α_education')

In [ ]:
az.summary(idata1b, var_names='α_education')

In [ ]:
az.summary(idata1, var_names='α_nationality')

In [ ]:
az.summary(idata1b, var_names='α_nationality')

Given the above, I hereafter drop the first model with the more naive priors and continue with the second model with the more informative priors.


Now to check the impact of each predictor on voter preference by examining the posterior distribution, its central tendency and spread and how it relates to the reference value of 0, which corresponds to to effect. Note that: 
* The orange number situates probabilistically the reference value (0=neither favorable nor unfavorable.)
* The black line corresponds to the 94% Highest Density Interval (HDI) - the narrowest interval containing 94% of the posterior distribution. 
    * The choice of 94% is a reminder that there is nothing magical about this number. Analysts should determine the appropriate size of the credibility interval that works best for their use case.  
    * This corresponds to a proabilistic statement; i.e. there is a 94% chance that given this data and this model, a voter correponding to this category would have an an offset within the HDI.

1. Baseline - average voter tendencies
    * On average there is a overwhelmingly positive outlook on EKRE, Eesti, Mitte, Reformierakond, and to a lesser extent, SDE. The average voter has also a high probability of being Undecided.
    * On the other hand, Isamaa, Parempoolsed, Rohelised and smaller parties grouped under the catch-all Other are not likely to be appealing to the average voter. 


In [ ]:
az.plot_posterior(idata1b, var_names='α', ref_val=0, textsize=16);

2. Education:
    * Basic Education: 
        * There is a very high probability that a voter belonging ot this category will be in favor of EKRE, Keskerakond, or Mitte
        * There is a very high probability against such a voter favoring Reformierakond, and to a lesser extent, SDE.
    * Secondary Education:
        * Voter have high probability of favoring EKRE and to a lesser extent Isamaa
        * Though not quite significant, voter are likely to be most hostile to SDE.
    * Higher Education:
        * Voters in this category are very hostile to EKRE, Mitte and Keskerakond and are unlikely to be undecided
        * Voters are quite sympathetic to Eesti and Reformierakond.


In [ ]:
az.plot_posterior(idata1b, var_names='α_education', ref_val=0, backend_kwargs=dict(tight_layout=True), textsize=20);

3. Nationality
* Estonians are: 
    * predominantly favorable to EKRE, Isamaa, Reformierakond;
    * largely unfavorable to Keskerakond and Mitte, and are less likely to be Undecided.
* Ethnic Russians (Other) on the other hand:
    * favor Keskerakond, Mitte, and are more likely to be Undecided;
    * are hostile to Reformierakond and to a lesser extent EKRE.


In [ ]:
f, axs = pp.subplots(ncols=3, nrows=8, figsize=(15, 40))
for ax in axs.ravel()[-2:]:
    ax.set_visible(False)
az.plot_posterior(idata1b, var_names='α_nationality', backend_kwargs=dict(tight_layout=True), ref_val=0, ax=axs, textsize=10);


#### Adding gender and age group
In accordance with the Bayesian workflow, the next step is to build a slightly more complex model, in this case by adding gender and vote age group as predictors to nationality and education, and see if can extract further insight. The model type is the same, just with a total of 4 offsets to the baseline instead of 2. I will also use stronger informative priors as I did in the second model that I ended up using for my analysis. Finally, the prior distribution for all coefficients is again the ZeroSumNormal distribution that I use to avoid non-identifiability due to overparameterization.
But first, to encode the additional predictors...


In [ ]:
gender_code, gender_label = data.gender.factorize(sort=True)
age_gp_code, age_group_label = data.age_group.factorize(sort=True)

In [ ]:
coords = {
    'age group': age_group_label,
    'education': education_label,
    'gender': gender_label,
    'nationality': nationality_label,
    'party choice': party_choice_label,
    'obs_idx': data.index
}

In [ ]:
def build_model2(coords, σ_baseline=1, σ_age=1, σ_edu=1, σ_gen=1, σ_nat=1):
    with pm.Model(coords=coords) as model2:
        age_gp_idx = pm.Data('age_gp', age_gp_code, dims='obs_idx')
        educ_idx = pm.Data('education', education_code, dims='obs_idx')
        gender_idx = pm.Data('gender', gender_code, dims='obs_idx')
        nat_idx = pm.Data('nationality', nationality_code, dims='obs_idx')
        party_choice_idx = pm.Data('party choice', party_choice_code, dims='obs_idx') 
        
        α_baseline = pm.ZeroSumNormal('α_baseline', sigma=σ_baseline, dims='party choice')
        α_age_group = pm.ZeroSumNormal('α_age_group', sigma=σ_age, dims=('age group', 'party choice'), n_zerosum_axes=2)
        α_education = pm.ZeroSumNormal('α_education', sigma=σ_edu, dims=('education', 'party choice'), n_zerosum_axes=2)
        α_gender = pm.ZeroSumNormal('α_gender', sigma=σ_gen, dims=('gender', 'party choice'), n_zerosum_axes=2)
        α_nationality = pm.ZeroSumNormal('α_nationality', sigma=σ_nat, dims=('nationality', 'party choice'), n_zerosum_axes=2)

        μ = α_baseline + α_age_group[age_gp_idx] + α_education[educ_idx] + α_gender[gender_idx] + α_nationality[nat_idx]
        p = pm.math.softmax(μ, axis=-1)

        _ = pm.Categorical('y', p=p, observed=party_choice_idx, dims='obs_idx')
        
    return model2

model2 = build_model2(coords=coords, σ_baseline=0.3, σ_age=0.4, σ_edu=0.3, σ_gen=0.2, σ_nat=0.2)
model2.to_graphviz()

In [ ]:
with model2:
    idata2 = pm.sample_prior_predictive()
    idata2.extend(pm.sample(chains=4, draws=2000, random_seed=42))
    idata2.extend(pm.sample_posterior_predictive(idata2))

In [ ]:
az.plot_trace(idata2, backend_kwargs=dict(tight_layout=True));

In [ ]:
az.summary(idata2, var_names='α_baseline')

In [ ]:
az.summary(idata2, var_names='α_age_group')

In [ ]:
az.summary(idata2, var_names='α_education')

In [ ]:
az.summary(idata2, var_names='α_gender')

In [ ]:
az.summary(idata2, var_names='α_nationality')

The fit took predictably longer; 2 minutes and 20 seconds on my macbook air m2. Moreover, 
* There were no divergences;
* Posteriors look sound and the traces exhibit good mixing for all chains;
* All $\hat{R}$ values were 1
* Bulk and tail ESS are about the same as the model with two predictors and relatively informative priors suggesting sampling efficiency has not deteriorated as a result of the increased model complexity.


In [ ]:
f, (axl, axr) = plot_forward_samples(idata2, categories=party_choice_label, categories_name='party choice', figsize=(12, 6))
axl.set_xticklabels(axl.get_xticklabels(), rotation=60);
axl.bar(x=axl.get_xticks(), height=mean_win.values,  color='k', fill=False, lw=3, width=1, zorder=13, label='data');
axr.set_xticklabels(axr.get_xticklabels(), rotation=60);